In [ ]:
using NamedTrajectories
using QuantumCollocation

using CairoMakie
using ForwardDiff
using LinearAlgebra
using Random

# Example III.1
-----
**How to build a quantum control problem**

Every quantum control problem requires a quantum system describing the dynamics, a goal, and time.

In [ ]:
system = QuantumSystem(0.01 * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])
U_goal = GATES[:X]

## Number of timesteps
T = 50

## Duration of timestep
Δt = 0.2

**NamedTrajectories..jl** stores the problem data. It has a lot of fields to help with this.

In [ ]:
NamedTrajectory |> fieldnames

In [ ]:
n_drives = length(system.H_drives)
a_bounds = fill(1.0, n_drives)
da_bounds = fill(1.0, n_drives)
dda_bounds = fill(1.0, n_drives)

## This will help us initialize the trajectory easily
traj = initialize_unitary_trajectory(
    U_goal,
    T,
    Δt,
    n_drives,
    (a = a_bounds, da = da_bounds, dda = dda_bounds)
)

The main thing a NamedTrajectory supports is indexing by symbols and plotting.

In [ ]:
## Inspect the control
traj.a

In [ ]:
plot(traj, [:a, :Ũ⃗])

Trajectories also enable simple construction of objectives.

In [ ]:
Q = 100.0
## Notice that we are using the iso_vec_operator symbol.
J = UnitaryInfidelityObjective(:Ũ⃗, traj, Q)

## Loss functions are evaluated on trajectories
Z⃗ = vec(traj)
J.L(Z⃗, traj)

Look at that! The infidelity loss is already zero. Are we done?

In [ ]:
R = 1e-2
J += QuadraticRegularizer(:a, traj, R)
J += QuadraticRegularizer(:da, traj, R)
J += QuadraticRegularizer(:dda, traj, R)

J.L(Z⃗, traj)

Let's add the dynamics constraints.

In [ ]:
## Integrators
integrators = [
    UnitaryPadeIntegrator(system, :Ũ⃗, :a, traj),
    DerivativeIntegrator(:a, :da, traj),
    DerivativeIntegrator(:da, :dda, traj)
]

In [ ]:
Random.seed!(1234)
ipopt_options = IpoptOptions(print_level=4, max_iter=50, recalc_y="yes", recalc_y_feas_tol=1e-2)

prob = QuantumControlProblem(
    system,
    traj,
    J,
    integrators,
    ipopt_options=ipopt_options,
)

In [ ]:
solve!(prob)
println("Unitary fidelity: ", unitary_fidelity(prob))

In [ ]:
plot(prob.trajectory, [:a, :Ũ⃗])

Of course, it is much easier to just call
```Julia
system = QuantumSystem(0.01 * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])
U_goal = GATES[:X]
T = 50
Δt = 0.2
problem = UnitarySmoothPulseProblem(system, U_goal, T, Δt)
solve!(problem)
```

# Exercises
-----

## Exercise III.1
**Inspect a gradient for correctness**

It is often the case that after we work very hard to solve for an analytic gradient, we need to make sure we did a good job coding it up. Not only do we need to profile the calculation to make sure our code is appropriately efficient, but we also need to be sure that our gradient was correct to begin with.

The next cell defines the following objective:
\begin{equation}
    J(\vec{\mathbf{Z}}) = ...
\end{equation}
Unfortunately, we made a mistake when writing our gradient.

Create a test trajectory, then use **ForwardDiff.jl** to help find the and fix the bug.

## Exercise III.2
**The proble template grand tour**